In [94]:
import pandas as pd

In [95]:
filepath = "../preprocessed_supermarket.xlsx"
df = pd.read_excel(filepath)

In [96]:
print(df.columns)

Index(['利润率', '制造商', '产品名称', '利润', '发货日期', '地区', '城市', '子类别', '折扣', '数量',
       '省/自治区', '类别', '细分', '订单日期', '邮寄方式', '销售额', '订单国家', '订单编号', '产品描述'],
      dtype='object')


## 数据预处理

### 商品单价计算

In [97]:
df['单价'] = df.apply(lambda row: row['销售额'] / ((1-row['折扣']) * row['数量']), axis=1)

In [98]:
# 计算每个商品的平均单价
avg_prices = df.groupby(['制造商', '产品名称'])['单价'].mean().reset_index()

# 将平均单价合并回原始DataFrame
df = pd.merge(df, avg_prices, on=['制造商','产品名称'], suffixes=('', '_平均'))

# 用平均单价替代原始的单价列
df['单价'] = df['单价_平均']

# 删除多余的列
df = df.drop(['单价_平均'], axis=1)

In [99]:
print(df[['制造商','产品名称', '单价']].drop_duplicates().head())

            制造商   产品名称          单价
0       Fiskars     剪刀  104.340789
33    GlobeWeis   搭扣信封   52.689550
51     Cardinal  孔加固材料   26.294440
83     Kleencut    开信刀  123.078807
101  KitchenAid    搅拌机  460.770714


### 商品成本计算

In [100]:
df['成本'] = df.apply(lambda row: (row['销售额'] - row['利润']) / row['数量'], axis=1)

In [101]:
# 计算每个商品的平均单价
avg_prices = df.groupby(['产品名称','制造商'])['成本'].mean().reset_index()

# 将平均单价合并回原始DataFrame
df = pd.merge(df, avg_prices, on=['产品名称','制造商'], suffixes=('', '_平均'))

# 用平均单价替代原始的单价列
df['成本'] = df['成本_平均']

# 删除多余的列
df = df.drop(['成本_平均'], axis=1)

In [102]:
print(df[['制造商', '产品名称', '成本']].drop_duplicates().head())

            制造商   产品名称          成本
0       Fiskars     剪刀   90.715705
33    GlobeWeis   搭扣信封   36.032937
51     Cardinal  孔加固材料   20.237016
83     Kleencut    开信刀   93.826235
101  KitchenAid    搅拌机  333.071905


### 各年销售情况

In [103]:
df['年份'] = df['订单日期'].dt.year
sales_count = df.groupby(['制造商', '年份', '产品名称', '产品描述'])['数量'].sum().reset_index()

print(sales_count)

       制造商    年份   产品名称 产品描述  数量
0     Acco  2013  孔加固材料   回收   3
1     Acco  2013  孔加固材料   实惠   8
2     Acco  2013  孔加固材料   耐用  13
3     Acco  2013    打孔机   回收   2
4     Acco  2013    打孔机   实惠   1
...    ...   ...    ...  ...  ..
5388   贝尔金  2016  闪存驱动器   耐用   2
5389   贝尔金  2016     鼠标  可编程   9
5390   贝尔金  2016     鼠标   回收   8
5391   贝尔金  2016     鼠标   实惠   8
5392   贝尔金  2016     鼠标   耐用   5

[5393 rows x 5 columns]


In [104]:
sales_count = sales_count.pivot_table(index=['制造商','产品名称','产品描述'], columns='年份', values='数量', fill_value=0)

# 将列名进行重命名
sales_count.columns = [f'{year}年卖出' for year in sales_count.columns]

# 将结果保存到新的DataFrame
sales_count = sales_count.reset_index()

# 输出结果
print(sales_count)

       制造商   产品名称 产品描述  2013年卖出  2014年卖出  2015年卖出  2016年卖出
0     Acco  孔加固材料   回收        3        9        0       18
1     Acco  孔加固材料   实惠        8        1        0       13
2     Acco  孔加固材料   耐用       13        0        0        7
3     Acco  孔加固材料   透明        0        0        5       24
4     Acco    打孔机   回收        2       15        0       26
...    ...    ...  ...      ...      ...      ...      ...
1948   贝尔金  闪存驱动器   耐用        6        0        0        2
1949   贝尔金     鼠标  可编程        1        3        7        9
1950   贝尔金     鼠标   回收        3        0        4        8
1951   贝尔金     鼠标   实惠        2        0        0        8
1952   贝尔金     鼠标   耐用        7        1        3        5

[1953 rows x 7 columns]


### 各年总利润

In [105]:
interest_count = df.groupby(['年份', '产品名称','产品描述'])['利润'].sum().reset_index()

print(interest_count)

        年份    产品名称  产品描述    利润
0     2013   个人复印机    彩色  1515
1     2013   个人复印机   数字化  2905
2     2013   个人复印机  每套两件   145
3     2013   个人复印机    红色    36
4     2013      书库    传统  1238
...    ...     ...   ...   ...
1594  2016  马尼拉纸信封    银色   465
1595  2016      鼠标   可编程 -1034
1596  2016      鼠标    回收    56
1597  2016      鼠标    实惠  1188
1598  2016      鼠标    耐用   964

[1599 rows x 4 columns]


In [106]:
interest_count = interest_count.pivot_table(index=['产品名称','产品描述'], columns='年份', values='利润', fill_value=0)
interest_count.columns = [f'{year}年利润' for year in interest_count.columns]
interest_count = interest_count.reset_index()

print(interest_count)

       产品名称  产品描述  2013年利润  2014年利润  2015年利润  2016年利润
0     个人复印机    彩色     1515     -899     5034     1661
1     个人复印机   数字化     2905      934     6620     1230
2     个人复印机  每套两件      145      382     2889     3591
3     个人复印机    红色       36     1578     1709     1855
4        书库    传统     1238    20288    11341    14887
..      ...   ...      ...      ...      ...      ...
403  马尼拉纸信封    银色      343      213      328      465
404      鼠标   可编程      -42       63      241    -1034
405      鼠标    回收      392     -126      264       56
406      鼠标    实惠      239      273      269     1188
407      鼠标    耐用       49      141      245      964

[408 rows x 6 columns]


## 有效信息合并

In [107]:
cols = ['产品名称','产品描述', '类别', '子类别', '制造商', '成本', '单价']
new_df = df[cols]
print(new_df)

     产品名称 产品描述    类别 子类别      制造商           成本           单价
0      剪刀   蓝色  办公用品  用品  Fiskars    90.715705   104.340789
1      剪刀   蓝色  办公用品  用品  Fiskars    90.715705   104.340789
2      剪刀  锯齿状  办公用品  用品  Fiskars    90.715705   104.340789
3      剪刀  锯齿状  办公用品  用品  Fiskars    90.715705   104.340789
4      剪刀   蓝色  办公用品  用品  Fiskars    90.715705   104.340789
...   ...  ...   ...  ..      ...          ...          ...
9930   木桌   黑色    家具  桌子    Bevis  1919.055556  2492.574074
9931   木桌   组装    家具  桌子    Lesro  1854.700000  2190.238889
9932   木桌   白色    家具  桌子    Lesro  1854.700000  2190.238889
9933   木桌   白色    家具  桌子    Lesro  1854.700000  2190.238889
9934  电脑桌   黑色    家具  桌子    Bevis  1892.384615  2307.794872

[9935 rows x 7 columns]


In [108]:
new_df = new_df.merge(sales_count, how='right')
new_df.drop_duplicates(inplace=True)
new_df = new_df.merge(interest_count, how='right')
new_df.drop_duplicates(inplace=True)
print(new_df)

       产品名称 产品描述  类别  子类别      制造商          成本          单价  2013年卖出  2014年卖出  \
0     个人复印机   彩色  技术  复印机  Brother  497.321905  665.129762        2        0   
1     个人复印机   彩色  技术  复印机  Hewlett  433.173485  673.141667        2        0   
2     个人复印机   彩色  技术  复印机       佳能  461.494762  666.330357        0        3   
3     个人复印机   彩色  技术  复印机       夏普  422.693972  558.507086       11        0   
4     个人复印机   彩色  技术  复印机       惠普  445.372727  578.890152        0        6   
...     ...  ...  ..  ...      ...         ...         ...      ...      ...   
1948     鼠标   耐用  技术   配件  Enermax  170.091349  185.816984        0        3   
1949     鼠标   耐用  技术   配件  Memorex  101.001247  130.952834        4        3   
1950     鼠标   耐用  技术   配件  SanDisk  130.276190  177.258377        2        0   
1951     鼠标   耐用  技术   配件       罗技   91.480392  136.840850        0        4   
1952     鼠标   耐用  技术   配件      贝尔金  145.953992  190.309290        7        1   

      2015年卖出  2016年卖出  2013年利润  2014年利

In [109]:
new_df.to_excel("products_info.xlsx", index=False)